# CityBikes

Send a request to CityBikes for the city of your choice. 

In [2]:
import requests
import pandas as pd

# Define the API endpoint URL
api_url = "https://api.citybik.es/v2/networks"
# Send a GET request to the API
response = requests.get(api_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Extract and print information from the response
    for network in data['networks']:
        name = network['name']
        city = network['location']['city']
        country = network['location']['country']
        print(f"Network: {name}\nCity: {city}\nCountry: {country}\n")
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")




Network: Velobike
City: Moscow
Country: RU

Network: Bysykkel
City: Bærum
Country: NO

Network: Bicincittà
City: Siena
Country: IT

Network: Cyclopolis
City: Maroussi
Country: GR

Network: Cyclopolis
City: Nafplio
Country: GR

Network: Bicincittà
City: Parco dei Colli di Bergamo
Country: IT

Network: Cyclopolis
City: Aigialeia
Country: GR

Network: Cyclopolis
City: Marathon
Country: GR

Network: Bicincittà
City: Mantova
Country: IT

Network: Cyclopolis
City: Νea Smyrni
Country: GR

Network: Cyclopolis
City: Μoschato-Tavros
Country: GR

Network: Cyclopolis
City: Αrxaia Olympia
Country: GR

Network: Cyclopolis
City: Kιato
Country: GR

Network: Bicincittà
City: Assemini
Country: IT

Network: Cyclopolis
City: Rhodes
Country: GR

Network: Cyclopolis
City: Florina
Country: GR

Network: Bicincittà
City: Tortolì
Country: IT

Network: Cyclopolis
City: Limnos
Country: GR

Network: Bicincittà
City: Gaeta
Country: IT

Network: Bicincittà
City: Borgarello
Country: IT

Network: Bicincittà
City: Pizz

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

Put your parsed results into a DataFrame.